In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
df=pd.read_csv("/kaggle/input/almond-types-classification/Almond.csv")

In [5]:
df.sample()

,Unnamed: 0,Length (major axis),Width (minor axis),Thickness (depth),Area,Perimeter,Roundness,Solidity,Compactness,Aspect Ratio,Eccentricity,Extent,Convex hull(convex area),Type
1400,1400,266.265533,133.462097,NaN,27344.0,758.700571,0.491069,0.955533,1.675209,1.995065,0.86531,0.739227,28616.5,MAMRA


In [6]:
df.isnull().sum()

Unnamed: 0                     0
Length (major axis)          857
Width (minor axis)           942
Thickness (depth)           1004
Area                           0
Perimeter                      0
Roundness                    857
Solidity                       0
Compactness                    0
Aspect Ratio                1799
Eccentricity                1799
Extent                         0
Convex hull(convex area)       0
Type                           0
dtype: int64

In [7]:
df.dtypes

Unnamed: 0                    int64
Length (major axis)         float64
Width (minor axis)          float64
Thickness (depth)           float64
Area                        float64
Perimeter                   float64
Roundness                   float64
Solidity                    float64
Compactness                 float64
Aspect Ratio                float64
Eccentricity                float64
Extent                      float64
Convex hull(convex area)    float64
Type                         object
dtype: object

In [8]:
df["Type"].unique()

array(['MAMRA', 'SANORA', 'REGULAR'], dtype=object)

In [30]:
X=df.drop(columns=["Unnamed: 0","Type"])
Y=df[["Type"]]

In [21]:
simple_impute=SimpleImputer(strategy="median")
x1=simple_impute.fit_transform(X)
X_df=pd.DataFrame(x1,columns=simple_impute.get_feature_names_out())

In [31]:
label_encoder=LabelEncoder()
Y["Type"]=label_encoder.fit_transform(Y)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/ipykernel_33/844936246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y["Type"]=label_encoder.fit_transform(Y)


In [32]:
Y

,Type
0,0
1,0
2,0
3,0
4,0
...,...
2798,2
2799,2
2800,2
2801,2


In [25]:
X_df.isnull().sum()


Length (major axis)         0
Width (minor axis)          0
Thickness (depth)           0
Area                        0
Perimeter                   0
Roundness                   0
Solidity                    0
Compactness                 0
Aspect Ratio                0
Eccentricity                0
Extent                      0
Convex hull(convex area)    0
dtype: int64

In [39]:
X_train,X_test,Y_train,Y_test=train_test_split(X_df,Y,test_size=0.2,random_state=42)

In [40]:
random_class=RandomForestClassifier(n_estimators=50)
random_class.fit(X_train,Y_train)

/tmp/ipykernel_33/783524776.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_class.fit(X_train,Y_train)


RandomForestClassifier(n_estimators=50)

In [41]:
random_class.score(X_test,Y_test)

0.8413547237076648

# # Quantum Machine Learning

In [43]:
!pip install cirq tensorflow_quantum

In [48]:
import cirq
import tensorflow_quantum as tfq
import tensorflow as tf
import sympy 
import numpy as np

2024-07-29 05:05:23.701835: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 05:05:23.702002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 05:05:23.869029: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [51]:
def create_quantum_circuits(X_train):
    qubits = [cirq.GridQubit(0, i) for i in range(X_train.shape[1])]
    circuits = []
    for x in X_train:
        circuit = cirq.Circuit()
        for i, value in enumerate(x):
            if value:
                circuit.append(cirq.X(qubits[i]))
        circuits.append(circuit)
    return circuits, qubits

circuits, qubits = create_quantum_circuits(X_train)
def create_model(qubits):
    # Define symbolic parameters
    symbols = sympy.symbols(f'theta0:{len(qubits)}')

    # Create parameterized circuit
    circuit = cirq.Circuit()
    for i, qubit in enumerate(qubits):
        circuit.append(cirq.rx(symbols[i])(qubit))
    
    readout_operators = [cirq.Z(qubit) for qubit in qubits]
    
    inputs = tf.keras.Input(shape=(), dtype=tf.dtypes.string)
    pqc = tfq.layers.PQC(circuit, readout_operators)(inputs)
    
    return tf.keras.Model(inputs=inputs, outputs=pqc)

model = create_model(qubits)
# _________________________
quantum_data = tfq.convert_to_tensor(circuits)  # Convert circuits to tensor
def hinge_loss(y_true, y_pred):
    return tf.reduce_mean(tf.maximum(1.0 - y_true * y_pred, 0.0))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=hinge_loss,metrics=['accuracy'])
history = model.fit(quantum_data, Y_train, epochs=100, verbose=1)
loss = model.evaluate(quantum_data, Y_train)
print('Loss:', loss)

IndexError: list index out of range

In [66]:
def create_quantum_circuits(X_train):
    circuits = []
    num_qubits = X_train.shape[1] # Number of qubits needed based on the number of features
    qubits = [cirq.GridQubit(0, i) for i in range(num_qubits)]  # Initialize qubits
    
    for _ in X_train:
        circuit = cirq.Circuit()
        for i, value in enumerate(X_train):
#             if value:  # Ensure there's a valid value to apply an operation
            circuit.append(cirq.X(qubits[i]))  # Adjust the gate based on the value
        circuits.append(circuit)
    return circuits, qubits
circuits, qubits = create_quantum_circuits(X_train)
def create_model(qubits):
    # Define symbolic parameters
    symbols = sympy.symbols(f'theta0:{len(qubits)}')

    # Create parameterized circuit
    circuit = cirq.Circuit()
    for i, qubit in enumerate(qubits):
        circuit.append(cirq.rx(symbols[i])(qubit))
    
    readout_operators = [cirq.Z(qubit) for qubit in qubits]
    
    inputs = tf.keras.Input(shape=(), dtype=tf.dtypes.string)
    pqc = tfq.layers.PQC(circuit, readout_operators)(inputs)
    
    return tf.keras.Model(inputs=inputs, outputs=pqc)

model = create_model(qubits)
# _________________________
quantum_data = tfq.convert_to_tensor(circuits)  # Convert circuits to tensor
def hinge_loss(y_true, y_pred):
    return tf.reduce_mean(tf.maximum(1.0 - y_true * y_pred, 0.0))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=hinge_loss,metrics=['accuracy'])
history = model.fit(quantum_data, Y_train, epochs=100, verbose=1)
# loss = model.evaluate(quantum_data, Y_

TypeError: Layer.add_weight() got multiple values for argument 'shape'

In [54]:
X_train.shape[1]

12

# # new Quantum Machine Learning

In [111]:
def create_circuit(X_train):
    qubits=[cirq.GridQubit(0,i) for i in range(X_train.shape[1])]
    circuits=[]
    for _ in X_train:
        circuit=cirq.Circuit()
        for i,value in enumerate(x):
            circuit.append(cirq.X(qubit[i]))
            circuits.append(circuit)
        return circuits,qubits
    circuits,qubits=create_circuit(X_train)
def create_model(qubits):
    symbols=sympy.symbols(f'theta0:{len(qubits)}')
    circuit=cirq.Circuit()
    for i,qubit in enumerate(qubits):
        circuit.append(cirq.rx(symbols[i])(qubit))
    read_out_operator=[cirq.Z(qubit) for qubit in qubits]
   
    inputs = tf.keras.Input(shape=(), dtype=tf.dtypes.string)
    pqc = tfq.layers.PQC(circuit, read_out_operator)(inputs)
    
    return tf.keras.Model(inputs=inputs, outputs=pqc)
    
model=create_model(qubits)
quantum_data=tfq.convert_to_tensor(circuits)
model.compile(optimizer="adam",metrics=["accuracy"])
model.fit(quantum_data,Y_train)
model.score(quantum_data,Y_test)

TypeError: Layer.add_weight() got multiple values for argument 'shape'

In [112]:
model

NameError: name 'model' is not defined

In [114]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy

# Define the quantum circuit with sympy symbols
qubits = [cirq.GridQubit(0, i) for i in range(3)]
symbols = sympy.symbols('theta0:3')
circuit = cirq.Circuit(
    cirq.H(qubits[0]),
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.Rz(symbols[0])(qubits[0]),
    cirq.Ry(symbols[1])(qubits[1]),
    cirq.Rx(symbols[2])(qubits[2])
)

# Define the readout operators
read_out_operator = [cirq.Z(qubit) for qubit in qubits]

def create_model(qubits):
    # Create the quantum circuit
    symbols = sympy.symbols('theta0:3')
    circuit = cirq.Circuit(
        cirq.H(qubits[0]),
        cirq.CNOT(qubits[0], qubits[1]),
        cirq.Rz(symbols[0])(qubits[0]),
        cirq.Ry(symbols[1])(qubits[1]),
        cirq.Rx(symbols[2])(qubits[2])
    )
    
    # Define the readout operators
    read_out_operator = [cirq.Z(qubit) for qubit in qubits]
    
    # Define the Keras input layer
    inputs = tf.keras.Input(shape=(), dtype=tf.dtypes.string)
    
    # Create the PQC layer
    pqc = tfq.layers.PQC(circuit, read_out_operator)(inputs)
    
    # Return the Keras model
    return tf.keras.Model(inputs=inputs, outputs=pqc)

model = create_model(qubits)
quantum_data = tfq.convert_to_tensor([circuit])
model.compile(optimizer="adam", metrics=["accuracy"])

# Dummy data for demonstration
x_train = quantum_data
y_train = tf.constant([[0], [1], [1]])

# Train the model
model.fit(x_train, y_train, epochs=1)


TypeError: Rz.__init__() takes 1 positional argument but 2 were given